In [ ]:
from datasets import load_dataset
import json

target_ds = load_dataset("singletongue/wikipedia-utils", "passages-c400-jawiki-20230403", split="train")

jsonl_path = 'wikipedia_passages_c400_jawiki_20230403.jsonl'
with open(jsonl_path, 'w', encoding='utf-8') as f:
    count = 0
    for item in target_ds:
        if count < 1025:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')
            count += 1
        else:
            break

In [ ]:
from datasets import load_dataset

target_ds = load_dataset('json', data_files="wikipedia_passages_c400_jawiki_20230403.jsonl", split="train")

for item in target_ds:
    print(item)

In [ ]:
from sentence_transformers import SentenceTransformer
from hf_hub_ctranslate2 import CT2SentenceTransformer


MODEL = CT2SentenceTransformer("/home/ubuntu/llm-rag-kit/multilingual-e5-large", compute_type="float16")

In [ ]:
from datasets.download import DownloadManager
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# データセットのロード
wikija_dataset = load_dataset('json', data_files="wikipedia_passages_c400_jawiki_20230403.jsonl", split="train")
index_local_path = "./outputs/faiss_indexes/multilingual-e5-large-passage/index_IVF256_PQ64.faiss"
# faiss index のロード
faiss_index = faiss.read_index(index_local_path)

In [ ]:
model = SentenceTransformer("/home/ubuntu/llm-rag-kit/multilingual-e5-large")
model.max_seq_length = 512

In [ ]:
def to_emb(model, text, prefix="query: "):
    return model.encode([prefix + text], normalize_embeddings=True)


emb = to_emb(model, "OpenAIに関して教えて?")

In [ ]:
TOP_K = 5
scores, indexes = faiss_index.search(emb, TOP_K)
for idx, (id, score) in enumerate(zip(indexes[0], scores[0])):
    data = wikija_dataset[int(id)]
    print((score, data["title"], data["text"][:100]))

poetry run streamlit run app.py